In [58]:
import os
import json
import random
# 设置路径
base_photo_src = "..\\data\\raw"
cam_dirs = [os.path.join(base_photo_src, f"cam{i}", "day") for i in range(1, 5)]
interpolation_dir = "..\\data\\interpolation_data"
output_path = os.path.join("..\\data", "dataset.json")           
len_of_timeseq = 3
# 取每个摄像头文件名并提取统一的 key（去掉前缀camX_）
cam_sets = []
for cam_dir in cam_dirs:
    cam_sets.append(set([x[5:] for x in os.listdir(cam_dir) if x.endswith('.jpg')]))

# 找到四个摄像头共有的图像
common_elements = set.intersection(*cam_sets)
print(f"共有图像: {common_elements}")



共有图像: {'2025-04-02_20_frame_009.jpg', '2025-04-27_14_frame_030.jpg', '2025-03-16_07_frame_004.jpg', '2025-04-14_06_frame_015.jpg', '2025-04-01_11_frame_025.jpg', '2025-03-27_10_frame_017.jpg', '2025-04-17_07_frame_012.jpg', '2025-03-16_16_frame_005.jpg', '2025-03-31_18_frame_022.jpg', '2025-03-30_19_frame_010.jpg', '2025-03-27_16_frame_002.jpg', '2025-03-27_18_frame_001.jpg', '2025-03-30_16_frame_006.jpg', '2025-04-05_06_frame_012.jpg', '2025-04-21_12_frame_005.jpg', '2025-04-03_18_frame_021.jpg', '2025-03-20_12_frame_013.jpg', '2025-03-18_07_frame_024.jpg', '2025-04-16_09_frame_013.jpg', '2025-04-08_20_frame_010.jpg', '2025-04-13_16_frame_014.jpg', '2025-03-29_13_frame_028.jpg', '2025-03-20_11_frame_012.jpg', '2025-03-26_20_frame_007.jpg', '2025-03-19_10_frame_016.jpg', '2025-03-26_12_frame_014.jpg', '2025-03-18_12_frame_005.jpg', '2025-04-19_06_frame_006.jpg', '2025-03-30_16_frame_002.jpg', '2025-04-02_18_frame_004.jpg', '2025-04-08_16_frame_015.jpg', '2025-04-14_07_frame_014.jpg', '

In [59]:
npy_dirs = os.listdir(interpolation_dir)
# 构建 JSON 数据
tmp_results = {}
timestamp_set = {}
for elem in sorted(common_elements):  # 如 '2025-03-26_18_frame_003.jpg'
    # 提取时间戳部分：2025-03-26_18_frame_003 -> 2025-03-26-18-003
    time_key = elem.replace('.jpg', '').replace('_', '-')
    time_key = time_key.replace('frame-', '')
    # 构建四个图像的完整路径
    image_paths = [
        os.path.join(cam_dirs[i], f"cam{i+1}_{elem}")
        for i in range(4)
    ]
    timestamp = time_key[:13]  # 提取时间戳部分
    # 构建浓度文件路径
    npy_path = f"{timestamp}.npy"

    # 只有在浓度文件存在时才加入
    if npy_path in npy_dirs:
        if timestamp not in timestamp_set:
            timestamp_set[timestamp] = []
            timestamp_set[timestamp].append(time_key)
        else:
            timestamp_set[timestamp].append(time_key)
        tmp_results[time_key] = image_paths
print(len(tmp_results))
# 打印前五个结果
print("前五个结果：")




11152
前五个结果：


In [ ]:
cnt_seq_time = 0
cnt = 0
results = {}
for i, (key,value) in enumerate(timestamp_set.items()):
   ready = True
   print(f"现在开始检查第{i}个时间戳：{key}")
   for j in range(len_of_timeseq):
      seq_key = key[:11] + str(int(key[11:13]) + j).zfill(2) #最后的数字向右补零对齐，2位数
      print(f"seq_key: {seq_key}")
      if seq_key not in timestamp_set:
         print(f"{key} 缺少第{j}个时间戳：{seq_key}")
         ready = False
         break
   if ready:
      for k in range(len_of_timeseq):
         seq_key = key[:11] + str(int(key[11:13]) + k).zfill(2)
         ip = timestamp_set[seq_key]
         for j in range(30):
            index = cnt + j
            if f"seq_{index}" not in results:
               results[f"seq_{index}"] = {}
            if f"{seq_key}.npy" not in results[f"seq_{index}"]:
               results[f"seq_{index}"][f"{seq_key}.npy"] = []
            if j < len(ip):
               results[f"seq_{index}"][f"{seq_key}.npy"].append(tmp_results[ip[j]])
            else:
               results[f"seq_{index}"][f"{seq_key}.npy"].append(tmp_results[ip[random.randint(0, len(ip)-1)]])
      cnt += 30
      cnt_seq_time += 1
      print(f"连续时间戳数量: {cnt_seq_time}")
#展示前五个results数据
print("前五个结果：")
for i, (key, value) in enumerate(results.items()):
    if i < 5:
        print(f"{key}: {value}")
# 将结果保存为 JSON 文件
with open(output_path, 'w') as f:
    json.dump(results, f, indent=4)



现在开始检查第0个时间戳：2025-03-07-10
seq_key: 2025-03-07-10
seq_key: 2025-03-07-11
2025-03-07-10 缺少第1个时间戳：2025-03-07-11
现在开始检查第1个时间戳：2025-03-07-13
seq_key: 2025-03-07-13
seq_key: 2025-03-07-14
2025-03-07-13 缺少第1个时间戳：2025-03-07-14
现在开始检查第2个时间戳：2025-03-16-06
seq_key: 2025-03-16-06
seq_key: 2025-03-16-07
seq_key: 2025-03-16-08
连续时间戳数量: 1
现在开始检查第3个时间戳：2025-03-16-07
seq_key: 2025-03-16-07
seq_key: 2025-03-16-08
seq_key: 2025-03-16-09
连续时间戳数量: 2
现在开始检查第4个时间戳：2025-03-16-08
seq_key: 2025-03-16-08
seq_key: 2025-03-16-09
seq_key: 2025-03-16-10
连续时间戳数量: 3
现在开始检查第5个时间戳：2025-03-16-09
seq_key: 2025-03-16-09
seq_key: 2025-03-16-10
seq_key: 2025-03-16-11
连续时间戳数量: 4
现在开始检查第6个时间戳：2025-03-16-10
seq_key: 2025-03-16-10
seq_key: 2025-03-16-11
seq_key: 2025-03-16-12
连续时间戳数量: 5
现在开始检查第7个时间戳：2025-03-16-11
seq_key: 2025-03-16-11
seq_key: 2025-03-16-12
seq_key: 2025-03-16-13
连续时间戳数量: 6
现在开始检查第8个时间戳：2025-03-16-12
seq_key: 2025-03-16-12
seq_key: 2025-03-16-13
seq_key: 2025-03-16-14
2025-03-16-12 缺少第2个时间戳：2025-03-16-14
现在开始检